install the required libraries:
spacy, spacy_fastlang

In [ ]:
!pip -q install spacy
!pip -q install spacy_fastlang

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import spacy_fastlang
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("language_detector")

can ignore the warning above

In [ ]:
text_content = 'काफी बढ़िया, इसी दौरान बदल फटने की न्यूज आई थी,  अमरनाथ धाम की यात्रा करवाने के लिए धन्यवाद।'
doc = nlp(text_content)
print(doc._.language, doc._.language_score)

hi 0.9663452506065369


check if the file is uploaded or not

In [ ]:
!ls

Ooty_excel.xlsx  sample_data


In [ ]:
import pandas as pd

comments = pd.read_excel('Ooty_excel.xlsx')

In [ ]:
nullvals = comments.isnull().any(axis = 1)

nullrows = comments[nullvals]
nullrows.shape

(4, 3)

4 rows are null. hence we remove them from our further process

In [ ]:
comments.dropna(inplace = True)

checking if we still have null values

In [ ]:
comments[comments.isnull().any(axis = 1)].shape

(0, 3)

hence, we successfully removed all the null values. We now filter off the non english comments. for that I will use spacy to detect language with a certain minimum language_score

In [ ]:
comments.head()

,idx,comment,tags
0,0,We went here for 4days...covered it via all d ...,personal experience
1,1,hi from South Africa my late husband and I wen...,personal experience
2,2,The best thing as a travel vlogger you have do...,informative
3,3,"Hi I am from hinganghat,Maharashtra.The knowle...",appreciation
4,4,Ooty enjoys a cool and pleasant climate round ...,general


We first detect the language we have as per spacy and add it as a different column 'lang_detected' with 'score'

In [ ]:
comments['lang_detected'] = comments.apply(lambda row : (nlp(row['comment'])._.language), axis = 1)
comments['score'] = comments.apply(lambda row : (nlp(row['comment'])._.language_score), axis = 1)

In [ ]:
comments.loc[comments['score'] > 0.9]

,idx,comment,tags,lang_detected,score
0,0,We went here for 4days...covered it via all d ...,personal experience,en,0.931457
1,1,hi from South Africa my late husband and I wen...,personal experience,en,0.995573
2,2,The best thing as a travel vlogger you have do...,informative,en,0.939251
3,3,"Hi I am from hinganghat,Maharashtra.The knowle...",appreciation,en,0.939377
4,4,Ooty enjoys a cool and pleasant climate round ...,general,en,0.946688
...,...,...,...,...,...
919,919,ooty looks overrated.,negative,en,0.981063
920,920,Dude it's not kannad its kannada pls know befo...,negative,en,0.968612
921,921,I dint like ooty though we stayed at savoy hot...,personal experience,en,0.933359
922,922,Ooty lake is so crowded with a long serpentine...,personal experience,en,0.974041


only 239 comments are tagged as 1 language with >90% accuracy. thus we need to improve our dataset to actually work. for this, we need to reduce our target accuracy and also transliterate some hinglish texts, which consists of the vast majority of the text, to hindi.

this will improve the overall confidence of dataset

lets try deep-translator, some new api idk

In [ ]:
!pip -q install -U deep-translator

In [ ]:
from deep_translator import GoogleTranslator as trsl

In [ ]:
cmmt = 'Very nice bhai Kuchh Bhi Kaho aapki Muskan Badi Pyari Hai'
inter = trsl(source = 'en', target = 'hindi').translate(cmmt)
print(inter, trsl(source = 'hindi', target = 'en').translate(inter))

वेरी नाइस भाई कुछ भी कहो आपकी मुस्कान बड़ी प्यारी है Very nice brother say anything your smile is very cute


basically for hinglish text, we need to keep source as en so that it translates to hindi. otherwise it detects source as hindi and does not translate at all

In [ ]:
comments.loc[comments['score'] > 0.8]

,idx,comment,tags,lang_detected,score
0,0,We went here for 4days...covered it via all d ...,personal experience,en,0.931457
1,1,hi from South Africa my late husband and I wen...,personal experience,en,0.995573
2,2,The best thing as a travel vlogger you have do...,informative,en,0.939251
3,3,"Hi I am from hinganghat,Maharashtra.The knowle...",appreciation,en,0.939377
4,4,Ooty enjoys a cool and pleasant climate round ...,general,en,0.946688
...,...,...,...,...,...
919,919,ooty looks overrated.,negative,en,0.981063
920,920,Dude it's not kannad its kannada pls know befo...,negative,en,0.968612
921,921,I dint like ooty though we stayed at savoy hot...,personal experience,en,0.933359
922,922,Ooty lake is so crowded with a long serpentine...,personal experience,en,0.974041


we now need to filter out the "hinglish" text, which would basically be detected as other languages erroneously.

In [ ]:
# comments.loc[comments['score'] > 0.7]

In [ ]:
comments.loc[comments['lang_detected'].isin(['hi', 'en'])]

,idx,comment,tags,lang_detected,score
0,0,We went here for 4days...covered it via all d ...,personal experience,en,0.931457
1,1,hi from South Africa my late husband and I wen...,personal experience,en,0.995573
2,2,The best thing as a travel vlogger you have do...,informative,en,0.939251
3,3,"Hi I am from hinganghat,Maharashtra.The knowle...",appreciation,en,0.939377
4,4,Ooty enjoys a cool and pleasant climate round ...,general,en,0.946688
...,...,...,...,...,...
919,919,ooty looks overrated.,negative,en,0.981063
920,920,Dude it's not kannad its kannada pls know befo...,negative,en,0.968612
921,921,I dint like ooty though we stayed at savoy hot...,personal experience,en,0.933359
922,922,Ooty lake is so crowded with a long serpentine...,personal experience,en,0.974041


we see that majority of our text is in english or hindi. lets now check how many arent in hindi or eng

In [ ]:
comments.loc[~comments['lang_detected'].isin(['hi', 'en'])]

,idx,comment,tags,lang_detected,score
44,44,Ooty jitna sundar hai...utna hi khoobsurat...I...,general,hr,0.230609
68,68,Beautiful!! Just Wow,general,ko,0.337835
126,126,i am your big fan... specially m ur big fan of...,appreciation,fy,0.252074
127,127,Bhai aap so good ..aapki smile so sweet,appreciation,nl,0.361531
129,129,"bhai ki smile dekhlo, din bn jata hai..bohot p...",appreciation,ro,0.487822
...,...,...,...,...,...
905,905,Waha pura saal thand parta hai kya??,informative,it,0.447462
907,907,hai,general,it,0.995315
908,908,Me gum ke aaya Hu bhai,personal experience,id,0.228986
911,911,படுக இன மக்களின் பாரம்பரியத்தை வெளிப்படுத்தும்...,general,ta,0.998860


lets check the languages that have been erroneously detected

In [ ]:
non_hinen_comments = comments.loc[~comments['lang_detected'].isin(['hi', 'en'])]
non_hinen_comments.groupby(by = 'lang_detected').count()

,idx,comment,tags,score
lang_detected,,,,
af,2,2,2,2
als,2,2,2,2
bh,1,1,1,1
br,3,3,3,3
ca,2,2,2,2
ceb,5,5,5,5
da,2,2,2,2
de,9,9,9,9
eo,3,3,3,3


from common sense, we cannot have languages such as italian, polish, russian or slovenian in our dataset. this is also shown by the low confidence scores. thus, we now check if any of these rows have actual comments which are non hindi/english using our confidence scores

In [ ]:
non_hinen_comments.loc[non_hinen_comments['score'] > 0.9]

,idx,comment,tags,lang_detected,score
205,205,Bohat sundar place hai Ooty,general,it,0.911680
241,241,Bike ride safe hai ooty may?,planning,it,0.930381
269,269,खूपच छान माहिती दिली आहे,appreciation,mr,0.991281
320,320,MERA DIL BHI KITNA PAGAL HAI\nYE PYAAR OOTY SE...,general,de,0.980222
433,433,1500 rs me hote room charges nahi hota hai jad...,informative,it,0.907851
660,660,Kya va per photograpar hota hai,informative,it,0.943127
850,850,Facebook par aapko dekhu,general,fr,0.975433
907,907,hai,general,it,0.995315
911,911,படுக இன மக்களின் பாரம்பரியத்தை வெளிப்படுத்தும்...,general,ta,0.998860


thus, we see that there is 1 marathi and 1 tamil comment which is actually judged properly. the rest are only misindentified texts. lets translate them to hindi so that we can finally move to our modelling phase

In [ ]:
trsl(source = 'mr', target = 'hindi').translate(non_hinen_comments.loc[269]['comment'])
# non_hinen_comments.loc[273]['comment']

'बहुत अच्छी जानकारी'

In [ ]:
non_hinen_comments = non_hinen_comments.loc[(non_hinen_comments['idx'] != 269) & (non_hinen_comments['idx'] != 911)].copy()

we translate the comments from target language as english to hindi, and store them in a new column

In [ ]:
non_hinen_comments['trans_comment'] = non_hinen_comments.apply(lambda row : trsl(source = 'en', target = 'hi').translate(row['comment']), axis = 1)

In [ ]:
non_hinen_comments

,idx,comment,tags,lang_detected,score,trans_comment
44,44,Ooty jitna sundar hai...utna hi khoobsurat...I...,general,hr,0.230609,ऊटी जितना सुंदर है... उतना ही खूबसूरत... मुझे ...
68,68,Beautiful!! Just Wow,general,ko,0.337835,सुंदर!! बस वाह
126,126,i am your big fan... specially m ur big fan of...,appreciation,fy,0.252074,मैं आपका बहुत बड़ा फैन हूं... खासतौर पर मैं आप...
127,127,Bhai aap so good ..aapki smile so sweet,appreciation,nl,0.361531,भाई आप सो गुड..आपकी स्माइल सो स्वीट
129,129,"bhai ki smile dekhlo, din bn jata hai..bohot p...",appreciation,ro,0.487822,"भाई की स्माइल देखो, दिन बन जाता है..बहोत प्यार..."
...,...,...,...,...,...,...
903,903,Bhai SIM konsa Chalta hai,informative,it,0.470113,भाई सिम कौनसा चलता है
905,905,Waha pura saal thand parta hai kya??,informative,it,0.447462,वहा पूरा साल ठंडा परता है क्या ??
907,907,hai,general,it,0.995315,है
908,908,Me gum ke aaya Hu bhai,personal experience,id,0.228986,में गम के आया हूं भाई


we now need to make a new df with the old comments (hindi and english combined) and add this new one

In [ ]:
comments.loc[comments['lang_detected'].isin(['hi', 'en', 'ta', 'mr'])]

,idx,comment,tags,lang_detected,score
0,0,We went here for 4days...covered it via all d ...,personal experience,en,0.931457
1,1,hi from South Africa my late husband and I wen...,personal experience,en,0.995573
2,2,The best thing as a travel vlogger you have do...,informative,en,0.939251
3,3,"Hi I am from hinganghat,Maharashtra.The knowle...",appreciation,en,0.939377
4,4,Ooty enjoys a cool and pleasant climate round ...,general,en,0.946688
...,...,...,...,...,...
919,919,ooty looks overrated.,negative,en,0.981063
920,920,Dude it's not kannad its kannada pls know befo...,negative,en,0.968612
921,921,I dint like ooty though we stayed at savoy hot...,personal experience,en,0.933359
922,922,Ooty lake is so crowded with a long serpentine...,personal experience,en,0.974041


we can see that there are still some comments which are written in romanized hindi. we need to translate them. one way to detect them is to pick the ones with less score in english

In [ ]:
hinenbn_comments = comments.loc[comments['lang_detected'].isin(['hi', 'en', 'bn'])].copy()

In [ ]:
hinenbn_comments.loc[(hinenbn_comments['score'] < 0.7) & (hinenbn_comments['lang_detected'] == 'en')]

,idx,comment,tags,lang_detected,score
41,41,Bro can you list out the places to visit.. & p...,general,en,0.698463
58,58,Very beautiful,general,en,0.488406
60,60,Well explained video..thanks,appreciation,en,0.496471
64,64,"apke sare travel vlogs mei ooty sbse acha,safe...",personal experience,en,0.564737
78,78,Good vlog sir..everything is covered..,appreciation,en,0.685437
...,...,...,...,...,...
904,904,Hindi ni smjhte kya yha log? Kis language me b...,informative,en,0.271349
906,906,"""KANNADA"""" not Kannad..",general,en,0.330394
909,909,Sir aap se bat kese kru no.. Mil skta h kya aap,general,en,0.286817
912,912,thoda baskva next karo,general,en,0.567938


we need to convert all of these to hindi, since these are mostly hinglish

In [ ]:
hinglish_comments = hinenbn_comments.loc[(hinenbn_comments['score'] < 0.7) & (hinenbn_comments['lang_detected'] == 'en')].copy()

In [ ]:
non_hinglish_comments = hinenbn_comments.loc[~((hinenbn_comments['score'] < 0.7) & (hinenbn_comments['lang_detected'] == 'en'))]

In [ ]:
hinglish_comments['trans_comment'] = hinglish_comments.apply(lambda row : trsl(source = 'en', target = 'hi').translate(row['comment']), axis = 1)

In [ ]:
hinglish_comments

,idx,comment,tags,lang_detected,score,trans_comment
9,9,Amazing Amarnath Yatra Vlog. Well captured and...,appreciation,en,0.661922,अद्भुत अमरनाथ यात्रा व्लॉग। अच्छी तरह से कब्जा...
24,24,"Honestly aap success ki jis uchayi par ho , us...",appreciation,en,0.241651,"ईमानदारी से आप सफलता की जिस उठाई पर हो, उसके ब..."
30,30,Have a safe journey bro ️,general,en,0.317216,आपकी यात्रा सुरक्षित हो भाई ️
43,43,Good job Sir. Very hard work.,appreciation,en,0.635351,अच्छा काम किया साहब। बहुत कठिन कार्य।
46,46,Thank you for the detailed information bhai.,appreciation,en,0.614247,विस्तृत जानकारी के लिए धन्यवाद भाई।
...,...,...,...,...,...,...
582,582,Bhaiya ji aap vlogs mst bnate ho aap k video S...,appreciation,en,0.365175,भैया जी आप व्लॉग एमएसटी बनते हो आप के वीडियो स...
584,584,Apne video k beach m bahut aacha song gaya….,appreciation,en,0.644615,अपने वीडियो के बीच में बहुत अच्छा गाना गया...।
586,586,tent fair kitna ka hay ?,informative,en,0.376918,टेंट फेयर कितना का है?
595,595,Is this solo journey,informative,en,0.635530,क्या यह सोलो जर्नी है


now all that is left is to combine the 3 seperated dataframes into one, i.e., non_hinglish_comments, hinglish_comments and non_hinen_comments


In [ ]:
non_hinglish_comments['trans_comment'] = non_hinglish_comments['comment'].copy()

<ipython-input-33-9fbcc6b6003c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_hinglish_comments['trans_comment'] = non_hinglish_comments['comment'].copy()


In [ ]:
non_hinglish_comments

,idx,comment,tags,lang_detected,score,trans_comment
0,0,We went here for 4days...covered it via all d ...,personal experience,en,0.931457,We went here for 4days...covered it via all d ...
1,1,hi from South Africa my late husband and I wen...,personal experience,en,0.995573,hi from South Africa my late husband and I wen...
2,2,The best thing as a travel vlogger you have do...,informative,en,0.939251,The best thing as a travel vlogger you have do...
3,3,"Hi I am from hinganghat,Maharashtra.The knowle...",appreciation,en,0.939377,"Hi I am from hinganghat,Maharashtra.The knowle..."
4,4,Ooty enjoys a cool and pleasant climate round ...,general,en,0.946688,Ooty enjoys a cool and pleasant climate round ...
...,...,...,...,...,...,...
919,919,ooty looks overrated.,negative,en,0.981063,ooty looks overrated.
920,920,Dude it's not kannad its kannada pls know befo...,negative,en,0.968612,Dude it's not kannad its kannada pls know befo...
921,921,I dint like ooty though we stayed at savoy hot...,personal experience,en,0.933359,I dint like ooty though we stayed at savoy hot...
922,922,Ooty lake is so crowded with a long serpentine...,personal experience,en,0.974041,Ooty lake is so crowded with a long serpentine...


now lets check the shapes of our newly made dataframe to check for data loss

In [ ]:
print('shape of comments ', comments.shape)
print('\nshape of hinglish_comments ', hinglish_comments.shape)
print('shape of non_hinglish_comments ', non_hinglish_comments.shape)
print('shape of non_hinen_comments ', non_hinen_comments.shape)

shape of comments  (920, 5)

shape of hinglish_comments  (334, 6)
shape of non_hinglish_comments  (381, 6)
shape of non_hinen_comments  (203, 6)


it seems that our new dataset is 2 away from the original dataset. we are probably missing out the marathi and tamil comment. since these are only 2 in number, we can safely ignore them :)

hence, we are ready to combine our transformed dataset for inputting the model. i would combine the dataset and save it in a file for easy access later

In [ ]:
combined = [hinglish_comments, non_hinglish_comments, non_hinen_comments]
result = pd.concat(combined)

In [ ]:
result

,idx,comment,tags,lang_detected,score,trans_comment
41,41,Bro can you list out the places to visit.. & p...,general,en,0.698463,भाई क्या आप घूमने की जगहों की सूची बना सकते है...
58,58,Very beautiful,general,en,0.488406,अति खूबसूरत
60,60,Well explained video..thanks,appreciation,en,0.496471,अच्छी तरह से समझाया गया वीडियो..धन्यवाद
64,64,"apke sare travel vlogs mei ooty sbse acha,safe...",personal experience,en,0.564737,"आपके सारे ट्रैवल व्लॉग मे ऊटी एसबीआई अच्छा, सु..."
78,78,Good vlog sir..everything is covered..,appreciation,en,0.685437,अच्छा व्लॉग सर..सब कुछ कवर किया गया है..
...,...,...,...,...,...,...
903,903,Bhai SIM konsa Chalta hai,informative,it,0.470113,भाई सिम कौनसा चलता है
905,905,Waha pura saal thand parta hai kya??,informative,it,0.447462,वहा पूरा साल ठंडा परता है क्या ??
907,907,hai,general,it,0.995315,है
908,908,Me gum ke aaya Hu bhai,personal experience,id,0.228986,में गम के आया हूं भाई


now we run the detect language and score function again to finally assign true values in our dataframe

In [ ]:
result[result.isnull().any(axis = 1)]

,idx,comment,tags,lang_detected,score,trans_comment


we see that there are no rows with null data! hence we dont need to drop anything

In [ ]:
result.dropna(inplace = True)

In [ ]:
result['lang_detected'] = result.apply(lambda row : (nlp(row['trans_comment'])._.language), axis = 1)
result['score'] = result.apply(lambda row : (nlp(row['trans_comment'])._.language_score), axis = 1)

In [ ]:
result.groupby(by = ['lang_detected']).count()

,idx,comment,tags,score,trans_comment
lang_detected,,,,,
en,355,355,355,355,355
hi,525,525,525,525,525
kk,1,1,1,1,1
mr,32,32,32,32,32
ne,2,2,2,2,2
sa,1,1,1,1,1
zh,2,2,2,2,2


we can be assured that our text is now completely in hindi and english and the other languages have been detected erroneously. i will now save the final dataset into an excel

In [ ]:
result.to_excel('Ooty_cleaned.xlsx')